In [ ]:
import os
import pandas as pd

dataset_path = "E:/ML_NLP/spam_ham_dataset" 

data = []  

# Проходим по папкам 1/ - 6/
for folder in os.listdir(dataset_path):  
    folder_path = os.path.join(dataset_path, folder)
    
    if os.path.isdir(folder_path):  
        for label in ['spam', 'ham']: 
            label_path = os.path.join(folder_path, label)
            
            if os.path.isdir(label_path):  
                for filename in os.listdir(label_path):  
                    file_path = os.path.join(label_path, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            text = f.read()  
                            data.append({'text': text, 'label': label})  
                    except Exception as e:
                        print(f"Ошибка с файлом {file_path}: {e}")

# Превращаем в DataFrame
df = pd.DataFrame(data)  
df.head()  


,text,label
0,Subject: dobmeos with hgh my energy level has ...,spam
1,Subject: your prescription is ready . . oxwq s...,spam
2,Subject: get that new car 8434\npeople nowthe ...,spam
3,"Subject: await your response\ndear partner ,\n...",spam
4,"Subject: coca cola , mbna america , nascar par...",spam
